# f_neuralNetwork
----

Written in the Python 3.7.9 Environment with the following package versions

    * joblib 1.0.1
    * numpy 1.19.5
    * pandas 1.3.1
    * scikit-learn 0.24.2
    * tensorflow 2.5.0

By Nicole Lund 

This Jupyter Notebook tunes a neural network model for Exoplanet classification from Kepler Exoplanet study data.

Column descriptions can be found at https://exoplanetarchive.ipac.caltech.edu/docs/API_kepcandidate_columns.html 

**Source Data**

The source data used was provided by University of Arizona's Data Analytics homework assignment. Their data was derived from https://www.kaggle.com/nasa/kepler-exoplanet-search-results?select=cumulative.csv

The full data set was released by NASA at
https://exoplanetarchive.ipac.caltech.edu/cgi-bin/TblView/nph-tblView?app=ExoTbls&config=koi

In [14]:
# Import Dependencies

# Plotting
%matplotlib inline
import matplotlib.pyplot as plt

# Data manipulation
import numpy as np
import pandas as pd
from statistics import mean
from operator import itemgetter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical


# Parameter Selection
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.experimental import enable_halving_search_cv  # noqa
from sklearn.model_selection import HalvingGridSearchCV

# Model Development
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.models import load_model

# Model Metrics
from sklearn.metrics import classification_report

# Save files
import joblib



# # Ignore deprecation warnings
# import warnings
# warnings.simplefilter('ignore', FutureWarning)

In [15]:
# Set the seed value for the notebook, so the results are reproducible
from numpy.random import seed
seed(1)

# Read the CSV and Perform Basic Data Cleaning

In [16]:
# Import data
df = pd.read_csv("b_source_data/exoplanet_data.csv")
# print(df.info())

# Drop columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop rows containing null values
df = df.dropna()

# Display data info
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6991 entries, 0 to 6990
Data columns (total 41 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   koi_disposition    6991 non-null   object 
 1   koi_fpflag_nt      6991 non-null   int64  
 2   koi_fpflag_ss      6991 non-null   int64  
 3   koi_fpflag_co      6991 non-null   int64  
 4   koi_fpflag_ec      6991 non-null   int64  
 5   koi_period         6991 non-null   float64
 6   koi_period_err1    6991 non-null   float64
 7   koi_period_err2    6991 non-null   float64
 8   koi_time0bk        6991 non-null   float64
 9   koi_time0bk_err1   6991 non-null   float64
 10  koi_time0bk_err2   6991 non-null   float64
 11  koi_impact         6991 non-null   float64
 12  koi_impact_err1    6991 non-null   float64
 13  koi_impact_err2    6991 non-null   float64
 14  koi_duration       6991 non-null   float64
 15  koi_duration_err1  6991 non-null   float64
 16  koi_duration_err2  6991 

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Select your features (columns)

These features were chosen in a_tree_feature_selection.ipynb and b_forest_feature_selections.ipynb

In [17]:
# Set features from Decision Tree and Random Forest assessment.
tree_features = ['koi_fpflag_nt', 'koi_fpflag_co', 'koi_fpflag_ss', 'koi_model_snr']
forest_features = ['koi_fpflag_co', 'koi_fpflag_nt', 'koi_fpflag_ss', 'koi_model_snr', 'koi_prad']
tree_forest_selected_features = set(tree_features + forest_features)
tree_forest_selected_features

{'koi_fpflag_co',
 'koi_fpflag_nt',
 'koi_fpflag_ss',
 'koi_model_snr',
 'koi_prad'}

In [18]:
# Split dataframe into X and y
X = df[tree_forest_selected_features]
y = df["koi_disposition"]
print(X.shape, y.shape)

(6991, 5) (6991,)


# Create a Train Test Split

Use `koi_disposition` for the y values

In [19]:
# Split X and y into training and testing groups
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42)

In [20]:
# Display training data
X_train.head()

,koi_fpflag_nt,koi_fpflag_co,koi_model_snr,koi_fpflag_ss,koi_prad
4954,0,0,7.8,0,2.29
4235,0,1,12.7,0,1.82
848,0,0,17.9,0,2.31
2874,0,1,43.3,1,23.81
3016,0,0,17.4,0,1.40


# Pre-processing

In [21]:
# Scale the data with MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [22]:
# One-Hot-Encode the y data

# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [23]:
print('Unique KOI Disposition Values')
print(y.unique())
print('-----------')
print('Sample KOI Disposition Values and Encoding')
print(y_test[:5])
print(y_test_categorical[:5])

Unique KOI Disposition Values
['CONFIRMED' 'FALSE POSITIVE' 'CANDIDATE']
-----------
Sample KOI Disposition Values and Encoding
4982    FALSE POSITIVE
4866         CANDIDATE
2934    FALSE POSITIVE
5007    FALSE POSITIVE
3869    FALSE POSITIVE
Name: koi_disposition, dtype: object
[[0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]]


# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [24]:
# Code was modified from sample code presented on
# https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/
# 
# Function to create model, required for KerasClassifier
def create_model(neurons=5):
	# create model
	model = Sequential()
	model.add(Dense(neurons, input_dim=X_train_scaled.shape[1], activation='relu'))
	model.add(Dropout(0.2))
	model.add(Dense(units=y_train_categorical.shape[1], activation='softmax'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

In [32]:
# Code was modified from sample code presented on
# https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/

# Use scikit-learn to grid search the batch size and epochs

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
# batch_size = [10, 20, 30]
# epochs = [100, 500, 1000]
# neurons = [5, 10, 15, 20, 25, 30, 35, 40]
batch_size = [10, 20]
epochs = [10, 50]
neurons = [3, 5]
param_grid = dict(batch_size=batch_size, epochs=epochs, neurons=neurons)

# Apply GridSearchCV
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train_scaled, y_train_categorical)

# summarize results
print("--------------------------")
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

--------------------------
Best: 0.751073 using {'batch_size': 20, 'epochs': 50, 'neurons': 5}
0.738606 (0.004258) with: {'batch_size': 10, 'epochs': 10, 'neurons': 3}
0.729818 (0.004046) with: {'batch_size': 10, 'epochs': 10, 'neurons': 5}
0.731249 (0.009316) with: {'batch_size': 10, 'epochs': 50, 'neurons': 3}
0.731657 (0.009289) with: {'batch_size': 10, 'epochs': 50, 'neurons': 5}
0.733906 (0.006565) with: {'batch_size': 20, 'epochs': 10, 'neurons': 3}
0.731862 (0.008031) with: {'batch_size': 20, 'epochs': 10, 'neurons': 5}
0.734927 (0.005492) with: {'batch_size': 20, 'epochs': 50, 'neurons': 3}
0.751073 (0.033653) with: {'batch_size': 20, 'epochs': 50, 'neurons': 5}


# Model Results when using features identified by tree and random forest classifiers
* Grid Definition: 
    * X = 
    * batch_size = 
    * epochs = 
    * neurons = 
* Grid run duration: 
* Grid Best Result: 

# Create and Train the Model - Neural Network

In [33]:
# Create model
model = Sequential()

# Define first layer
model.add(Dense(units=5,
                activation='relu', input_dim=X_train_scaled.shape[1]))

# Define output layer
model.add(Dense(units=y_train_categorical.shape[1], activation='softmax'))    

# Review Model
print(model.summary())

# Compile Model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train model
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=50,
    batch_size=20,
    shuffle=True,
    verbose=0
)

# Evaluate the model using the testing data
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 5)                 30        
_________________________________________________________________
dense_9 (Dense)              (None, 3)                 18        
Total params: 48
Trainable params: 48
Non-trainable params: 0
_________________________________________________________________
None
66/66 - 0s - loss: 0.4635 - accuracy: 0.7398
Loss: 0.46345773339271545, Accuracy: 0.7397521734237671


# Save the Model

In [34]:
# Save the model
model.save("f_neuralNetwork_treeFeatures_model.h5")

In [35]:
# # Save the grid
joblib.dump(grid, './f_neuralNetwork_treeFeatures_grid.sav')

TypeError: can't pickle weakref objects